## Setup

### Install packages

In [ ]:
!pip install geopandas

In [ ]:
!pip install rasterio

In [3]:
import geopandas as gpd
import pandas as pd
import rasterio as rio
import ee
from google.colab import drive, auth
from shapely.geometry import Point, Polygon
import numpy as np
from numpy.random import normal
import json

### Authenticate

In [4]:
auth.authenticate_user()

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=nt9m_Pu4mFhvpsK1BxgLUVWD7hAaOEAGVqi5rXDshlE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgmOr5gPqTqG6H_D2wCUkU2YEsOm_ihrzx0XsGgorn0BLEYHQNpIgk

Successfully saved authorization token.


## Clean and Harmonize solar arrays


### Create GeoDataFrame with Annual Polygons
In this step we clean and improve the original data produced from our AI output. Assuming any arrays detected at time t should be present at time t+1, we iteratively dissolve overlaping polygons in each year, and then dissolve the resulting polygons with overlapping polygons in the following year. This fills in gaps to ensure that polygon Sij in year j = n at location i contains all polygons Sij for j<n. The resulting GeoDataFrame includes rows for each solar array in each year that it was present (array x year).

In [7]:
def eliminate_overlap(gdf, group):
  """Eliminate overlapping polygons within a group
  Args:
    gdf (GeoDataFrame):
    group (str): column name contianing group labels
  Return:
    GeoDataFrame:
  """
  dissolved = gdf.dissolve(group, as_index = False)
  exploded = dissolved.explode('geometry', True)
  return(exploded)


In [ ]:
def harmonize_years(gdf, group):
  """Iterate over a geodataframe by group eliminating overlapping polygons

  This funciton is analagous to a groupby and reduction with groups acting as elements
  provided to the reducer. Iterate over levels of a grouping variable, dissolving polygons 
  in the next group with output from the previous iteration.
  Args:
    gdf (GeoDataFrame): polygons over which to iterate. must contain a grouping variable
    group (str): column name of grouping variable.
  Returns: 
    GeoDataFrame
  """ 
  groups = gdf[group].unique()
  groups.sort()
  print(groups)
  gdflist = []
  for i, g in enumerate(groups):
    current = gdf[gdf[group] == g]
    if i == 0:
      eliminated = eliminate_overlap(current, group)
      gdflist.append(eliminated)
    else:
      combined = current.append(gdflist[-1])
      dissolved = combined.dissolve()
      exploded = dissolved.explode('geometry', True)
      exploded['system:time_start'] = g
      gdflist.append(exploded)
  gdf_final = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index = True), crs = gdflist[0].crs)
  return gdf_final


In [8]:
# read our per-state data from GEE into GeoDataFrames
fs = ['NY_solar', 'DE_solar', 'PA_solar', 'VA_solar', 'MD_solar']
files = [f'/content/drive/MyDrive/{f}.geojson' for f in fs]
gdfs = [gpd.read_file(f, driver = 'GeoJON') for f in files]
gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index = True), crs = gdfs[0].crs)

In [10]:
len(gdf)

2527

In [ ]:
annualGDF = harmonize_years(gdf, 'system:time_start')

['2017-06-01' '2018-06-01' '2019-06-01' '2020-06-01' '2021-06-01']


In [ ]:
annualGDF.head()
# print(len(final))

In [ ]:
# check that the number of arrays each year make sense (i.e should be increasing)
annualGDF.groupby('system:time_start').size()

system:time_start
2017-06-01    111
2018-06-01    327
2019-06-01    596
2020-06-01    765
2021-06-01    938
dtype: int64

In [ ]:
# finally, add a 'year' column
annualGDF['year'] = final.apply(lambda x: int(x['system_tim'][0:4]), axis = 1)

In [ ]:
# write data to GCS for ingestion to GEE
outFile = 'CPK_solarJun21_annual'
dstFile = 'gs://cic-solar-mevans'
annualGDF.to_file(outFile+'.shp')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
!gsutil mv {outFile}* {dstFile}

In [ ]:
!earthengine upload table --asset_id projects/mevans-cic-solar/assets/CPK_solarJun21_annual {dstFile+'/'+outFile+'.shp'}

Started upload task with ID: A5Y3UW2JDVSF5K4HCDZNANYE


In [ ]:
del gdf

### Create Dataframe with unique polygons per year
For statistical analysis, a GeoDataFrame with the same solar array represented in multiple years leads to pseudoreplication. Therefore, we need an alternatively structure dataset in which each row represents a solar array polygon in the first year that it appeared.

In [ ]:
# get the year of all overlapping polygons
joined = gpd.sjoin(solarGDF, solarGDF[['geometry', 'year']],'left','intersects')
# aggreate overlapping polygons retaining minimum year (i.e. year of appearance)
dissolved = joined.dissolve(by = joined.index, aggfunc = 'min')
# aggregate polygons per year of appearance
dissolved2 = dissolved.dissolve(by = 'year_right', aggfunc = 'first', as_index = False)
# convert multipart to single polygon geometries
exploded = dissolved2.explode(ignore_index = True, index_parts = False)

In [ ]:
outFile = 'CPK_solarJun21_firstyear'
dstFile = 'gs://cic-solar-mevans'
exploded.to_file(outFile+'.shp')

In [ ]:
!gsutil mv {outFile}* {dstFile}

In [ ]:
!earthengine upload table --asset_id projects/mevans-cic-solar/assets/CPK_solarJun21_firstyear {dstFile+'/'+outFile+'.shp'}

## Sampling

In [ ]:
STATES = ee.FeatureCollection("TIGER/2018/States")
states = ['Pennsylvania', 'Virginia', 'New York', 'Delaware', 'Maryland']
studyArea = STATES.filter(ee.Filter.inList('NAME', states))
TRACTS = ee.FeatureCollection("TIGER/2010/Tracts_DP1")
tracts = TRACTS.filterMetadata('aland10', 'greater_than', 10)

### Random polygons
We create random polygons to represent areas that could have been, but were not, developed for solar energy. Random polygons are 2:1 rectangles with size drawn from a normal distribution based on the observed mean and variance in mapped solar array sizes.

In [ ]:
# we will generate random 
def calc_area_sides(ft):
  """Estimates the side of a GEE Feature based on its area, assuming a 2:1 rectangle
  Params:
   ft (ee.Feature)
  Returns:
    ee.Feature: input features with additional attributes 'area' and 'side'
  """
  area = ft.area()
  side = area.divide(2).sqrt().divide(2)
  return ft.set('area', area, 'side', side)

solar_arrays = ee.FeatureCollection('projects/mevans-cic-solar/assets/CPK_solarJun21_firstyear').map(calc_area_sides)

In [ ]:
solar_arrays.filterMetadata('system_tim', 'equals', '2021-06-01').size().getInfo()

938

In [ ]:
# we want random polygons in proportion to real solar arrrays per state
def generate_random_perstate(state):
  """Generates 5x random points for the solar arrays within each state
  Args:
    state (str): TIGER name of state
  Returns:
    ee.FeatureCollection: features representing random points
  """
  st = studyArea.filterMetadata('NAME', 'equals', state)
  aoi = tracts.filterBounds(st)
  array_subset = solar_arrays.filterBounds(st).filterMetadata('system_tim', 'equals', '2021-06-01')
  narrays = array_subset.size()
  random = ee.FeatureCollection.randomPoints(aoi, narrays.multiply(5))
  return random


In [ ]:
randFtCol = studyArea.map(generate_random_perstate)#ee.FeatureCollection(ee.List(states).iterate(generate_random_perstate, ee.FeatureCollection([])))

In [ ]:
mdRandom = generate_random_perstate('Maryland')
deRandom = generate_random_perstate('Delaware')
vaRandom = generate_random_perstate('Virginia')
paRandom = generate_random_perstate('Pennsylvania')
nyRandom = generate_random_perstate('New York')

In [ ]:
randomFtCol = mdRandom.merge(deRandom).merge(vaRandom).merge(nyRandom).merge(paRandom)

In [ ]:
# Get the JSON objects for random points so we can build random rectangels using GeoPandas
crs = randomFtCol.geometry().projection().crs().getInfo()
randomPts = randomFtCol.geometry().getInfo()

In [ ]:
len(randomPts['coordinates'])

4595

In [ ]:
shapelyPts = [Point(coords) for coords in randomPts['coordinates']]

In [ ]:
randomGdf = gpd.GeoDataFrame(geometry = shapelyPts, crs = crs)

In [ ]:
# Alternatively, export our random points 
task = ee.batch.Export.table.toDrive(collection=randomFtCol, description='randompoints', fileFormat = 'GeoJSON')
task.start()

In [ ]:
randomGdf = gpd.read_file('/content/drive/MyDrive/randompoints.geojson', driver = 'GeoJSON')

In [ ]:
randomGdf.head()

In [ ]:
def make_random_rectangles(gdf, loc, scale):
  """Construct 2y x y and y x 2y rectangles at a location with y randomly determined by mean and std
  Args:
    gdf (GeoDataFrame): Points at which rectangles will be created
    loc (float): mean size of short side of rectangle
    scale (float): std of size of short side of rectangle
  Returns: 
    GeoDataFrame: randomly sized rectangles at input points
  """
  # create an array of random radii based on input mean and std
  geom = gdf.geometry.to_crs('epsg:3857')
  x = geom.x
  y= geom.y
  random_radii = normal(loc = loc, scale = scale, size = len(gdf))
  # this construct alternately creates 2y x y and y x 2y rectangles
  rectangles = [Polygon([[x[i]+(i%2 +1)*random_radii[i], y[i]+(2-i%2)*random_radii[i]], [x[i]+(i%2 +1)*random_radii[i],  y[i]-(2-i%2)*random_radii[i]], [x[i]-(i%2 +1)*random_radii[i], y[i]-(2-i%2)*random_radii[i]], [x[i]-(i%2 +1)*random_radii[i], y[i]+(2-i%2)*random_radii[i]], [x[i]+(i%2 +1)*random_radii[i], y[i]+(2-i%2)*random_radii[i]]]) for i in range(len(random_radii))]
  return gpd.GeoDataFrame(geometry = rectangles, crs = 'epsg:3857')

In [ ]:
# we may want to keep random polygons in GEE, so define an equivalent method
def make_random_circles_ee(ftCol, loc, scale):
  """Construct a rectangle at a centroid with random width and height within a specified range
  Args:
    gdf:
    loc:
    scale:
  Return:
  """
  # create an array of random radii based on input mean and std
  def random_radii(ft):
    radii = normal(loc = loc, scale = scale, size = 1)[0]
    return ft.buffer(radii)

  buffered = ftCol.map(random_radii)
  return buffered

In [ ]:
# calculate the mean and std of the short side of rectangles with area equal to mapped solar arrays
side_mean = solar_arrays.aggregate_mean('side').getInfo()
side_sd = solar_arrays.aggregate_sample_sd('side').getInfo()

In [ ]:
print(side_sd)

46.85494095575329


In [ ]:
# generate random rectangles at our random points
rectanglesGdf = make_random_rectangles(randomGdf, side_mean, side_sd)
len(rectanglesGdf) == len(randomGdf)

In [ ]:
# convert rectangles to GEE FeatureCollection and assign year = 2030 to distinguish from real arrays
rectanglesJson = json.loads(rectanglesGdf.geometry.to_crs('epsg:4326').to_json())
rectanglesFtCol = ee.FeatureCollection(rectanglesJson).map(lambda x:x.set('year', '2030'))

In [ ]:
task = ee.batch.Export.table.toAsset(collection =rectanglesFtCol, description = 'random rectangles', assetId = 'projects/mevans-cic-solar/assets/random_arrays' )
task.start()

### Sample raster data
We use GEE to efficiently sample covariates at real and random solar arrays that are represented as rasters

In [ ]:
NLCD = ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL")
SRTM = ee.Image("USGS/SRTMGL1_003")
STATES = ee.FeatureCollection("TIGER/2018/States")
DEM = ee.Image("USGS/3DEP/10m")
CDL = ee.ImageCollection("USDA/NASS/CDL")
randomArrays = ee.FeatureCollection("projects/mevans-cic-solar/assets/random_arrays")
arrays = ee.FeatureCollection("projects/mevans-cic-solar/assets/Outputs/CPK_solarJun21_firstyear")

In [ ]:
# create predictor variables based on NLCD 2016 data
nlcd16 = ee.Image('USGS/NLCD_RELEASES/2016_REL/2016').clip(studyArea)

landcover = nlcd16.select('landcover')

# percent impervious surface per pixel
impervious = nlcd16.select('impervious')

# percent tree cover per pixel
tree_cover = nlcd16.select('percent_tree_cover')

cdl = ee.Image(CDL.filterBounds(studyArea).filterDate('2016-01-01', '2016-12-31').first()).clip(studyArea)

# 0-1 indicator of agriculture per pixel
ag = cdl.select('cultivated').subtract(0)

# 0-1 indicator of non-agricultural 'open' land (e.g. fields, lawns, etc.)
open = landcover.eq(52).Or(landcover.eq(71)).Or(landcover.eq(21)).Or(landcover.eq(22));

# 10 m slope
dem = DEM.clip(studyArea)
slope = ee.Terrain.slope(dem).rename('slope')

In [ ]:
# combine predictor variables into single ee.Image
data = slope.addBands(ag).addBands(tree_cover).addBands(impervious).addBands(open)

# sample raster data at solar arrays
Ns = arrays.size().getInfo()
solarList = arrays.toList(Ns)

print(Ns)
x = 0
# while x < Ns:
  # subset = ee.FeatureCollection(solarList.slice(x, x+500))
arraySample = data.reduceRegions(
  collection= arrays,
  reducer= ee.Reducer.mean(),
  scale= 10,
  tileScale= 12
)

# save raster predictor data 
task1 = ee.batch.Export.table.toAsset(
  collection= arraySample,
  assetId = 'projects/mevans-cic-solar/assets/Outputs/CPK_arrays_covariates',
  description= 'CPK_arrays_covariates'
)

task1.start()
  
  # x+=500
# and export to Drive
task2 = ee.batch.Export.table.toDrive(
    collection = arraySample,
    description = 'CPK_arrays_covariates',
    fileFormat = 'GeoJSON'
)

task2.start()

953


In [ ]:
# sample raster data at random polygons
Nr = randomArrays.size().getInfo()
randomList = randomArrays.toList(Nr)

print(Nr)
# x = 0
# while x < Nr:
  # subset = ee.FeatureCollection(randomList.slice(x, x+500))
randomSample = data.reduceRegions(
  collection= randomArrays,
  reducer= ee.Reducer.mean(),
  scale= 10,
  tileScale= 12
)

task3 = ee.batch.Export.table.toAsset(
  collection= randomSample,
  assetId = 'projects/mevans-cic-solar/assets/CPK_random_covariates',
  description= 'CPK_random_covariates'
)

task3.start()

# and export to Drive
task4 = ee.batch.Export.table.toDrive(
    collection = randomSample,
    description = 'CPK_random_covariates',
    fileFormat = 'GeoJSON'
)

task4.start()
  # x+=500

4595


In [ ]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1645584335331,
 'description': 'CPK_random_GEEcovariates',
 'id': 'HX52K7GIZRH7MWP6X576ZDTH',
 'name': 'projects/earthengine-legacy/operations/HX52K7GIZRH7MWP6X576ZDTH',
 'start_timestamp_ms': 1645584373534,
 'state': 'RUNNING',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1645587223652}